(c) 2024 Manuel Razo. This work is licensed under a [Creative Commons
Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).
All code contained herein is licensed under an [MIT
license](https://opensource.org/licenses/MIT).

In [49]:
# Import project package
import Antibiotic

# Import package to handle DataFrames
import DataFrames as DF
import CSV

# Import library for Bayesian inference
import Turing
import MCMCChains
import FillArrays

# Import library to list files
import Glob

# Import packages to work with data
import DataFrames as DF

# Load CairoMakie for plotting
using CairoMakie
import PairPlots
import ColorSchemes

# Import basic math libraries
import LsqFit
import StatsBase
import LinearAlgebra
import Random
import Distances
import Distributions

# Activate backend
CairoMakie.activate!()

# Set PBoC Plotting style
Antibiotic.viz.theme_makie!()

# Bayesian Inference of $IC_{50}$ values

In this notebook, we will perform Bayesian inference on the $IC_{50}$ values of
the antibiotic resistance landscape. For this, we will use the raw `OD620`
measurements provided by Iwasawa et al. (2022).

Let's begin by loading the data into a DataFrame.

In [ ]:
# Load data into a DataFrame
df = CSV.read(
    "$(git_root())/data/Iwasawa_2022/iwasawa_tidy.csv", DF.DataFrame
)

first(df, 5)

To double-check that the structure of the table makes sense, let's plot the time
series for one example to see if the sequence agrees with the expectation.

In [ ]:
# Define data to use
data = df[
    (df.antibiotic.=="KM").&(df.env.=="Parent_in_KM").&(df.strain_num.==13).&.!(df.blank).&(df.concentration_ugmL.>0),
    :]
# Remove blank measurement
# Group data by day
df_group = DF.groupby(data, :day)

# Initialize figure
fig = Figure(size=(500, 300))

# Add axis
ax = Axis(
    fig[1, 1],
    xlabel="antibiotic concentration",
    ylabel="OD₆₂₀",
    xscale=log2
)

# Define colors for plot
colors = get(ColorSchemes.Blues_9, LinRange(0.25, 1, length(df_group)))

# Loop through days
for (i, d) in enumerate(df_group)
    # Sort data by concentration
    DF.sort!(d, :concentration_ugmL)
    # Plot scatter line
    scatterlines!(
        ax, d.concentration_ugmL, d.OD, color=colors[i], label="$(first(d.day))"
    )
end # for

# Add legend to plot
fig[1, 2] = Legend(
    fig, ax, "day", framevisible=false, nbanks=3, labelsize=10
)

fig

The functional form used by the authors to fit the data is
$$
f(x) = \frac{a}
{1+\exp \left[b\left(\log _2 x-\log _2 \mathrm{IC}_{50}\right)\right]} + c
\tag{1}
$$
where $a$, $b$, and $c$ are nuisance parameters of the model, $\mathrm{IC}_{50}$
is the parameter of interest, and $x$ is the antibiotic concentration. We can
define a function to compute this model.

In [ ]:
@doc raw"""
    logistic(logx, a, b, c, logic50)

Compute the logistic function used to model the relationship between antibiotic
concentration and bacterial growth, using log2 inputs for concentration and
IC₅₀.

This function implements the following equation:

f(x) = a / (1 + exp(b * (log(x) - log(IC₅₀)))) + c

# Arguments
- `logx`: log of the antibiotic concentration (input variable)
- `a`: Maximum effect parameter (difference between upper and lower asymptotes)
- `b`: Slope parameter (steepness of the curve)
- `c`: Minimum effect parameter (lower asymptote)
- `logic50`: log of the IC₅₀ parameter

# Returns
The computed effect (e.g., optical density) for the given log₂ antibiotic
concentration and parameters.

Note: This function is vectorized and can handle array inputs for `log2x`.
"""
function logistic(logx, a, b, c, logic50)
    return @. a / (1.0 + exp(b * (logx - logic50))) + c
end

## Bayesian definition of the problem

For our particular problem in question we have our variate—-the
log-concentration $x$--and our covariate--the log-optical density $y$. In
this context if we were to write a condition likelihood it would take the form
$$
\pi(y \mid f(x), x),
\tag{2}
$$
where $f(x)$ is a function that depends on the variates. A process is define as
setting a probability distribution over functions rather than over numbers.
Formally this becomes an infinite-dimensional Hilbert space of functions, or as
I like to thing about it: there are infinite possible functional relationships
between variates and covariates. The question is then how to set a probability
distribution over this infinite dimensional space? We can limit ourselves to a
subset of possible functions. In particular a Gaussian process, just as a
regular Gaussian distribution, is defined by a mean function $m$ and a
covariance function $k$, i.e.
$$
f \sim \mathcal{GP}(m, k).
\tag{3}
$$

What this means is that given our alele frequency measurements $y$ measured at
time $t$ we want to compute $f(t)$. For this we write Bayes theorem as
$$
\pi(f(x_*) \mid y, x_*) = 
\frac{\pi(y \mid f(x_*), x_*) \pi(f(x_*) \mid x_*)}{\pi(y \mid x_*)},
\tag{4}
$$
where $x_*$ indicates that we **sampled a finite number of concentrations**.

### Mean and covariance functions

The mean function $m(x)$ defines the basal value to which all realizations of
the Gaussian process will converge. In other words, the average of many many
samples of the Gaussian process would converge to this value. As it is common
practice, given the lack of *a priori* information of such convergence value, we
will set $m(x)$ to be zero.

The covariance function, also known as covariance kernel $k(x_1, x_2)$, sets the
variation of the Gaussian process around the mean function. Larger covariance
between two variates (time points) $x_1$ and $x_2$, means that their outputs
$f(x_1)$ and $f(x_2)$ cannot vary that much between them. We then expect that
the covariance between two time points should decay as the time gap between them
increases. From the several possible covariance functions that are commonly use
in practice we will focus on the so-called *exponential quadratic* kernel,
$$
k(x_1, x_2) = \alpha^2 \exp \left(-\frac{(x_1 - x_2)^2}{\rho^2} \right).
\tag{5}
$$
Here $\alpha$, known as the *marginal standard deviation*, controls the expected
variation in the variate output, while $\rho$, known as the *length scale* (or
in our specific case the time scale), sets the scale by which the variate
outputs change with respect to the distance between covariates. Intuitively one
can think of $\alpha$ as controlling the amplitude of the wiggles in the
function, and $\rho$ controls the frequency of such wiggles. These are known as
the **hyperparameters**, and basically allow this non-parametric inference to be
fine-tuned.

Let's define the covariance function.

In [ ]:
@doc raw"""
    cov_exp_quad(x, α, ρ; offset=1e-10)

Compute the Exponentiated Quadratic (RBF) covariance matrix.

# Arguments
- `x::AbstractVector`: Points where to evaluate the kernel.
- `α::Real`: Marginal standard deviation (output scale).
- `ρ::Real`: Length-scale parameter.

# Optional Keyword Arguments
- `offset::Real=1e-10`: Small value added to the diagonal for numerical
  stability.

# Returns
- `Matrix`: The computed covariance matrix.

# Description
This function computes the Exponentiated Quadratic (also known as Radial Basis
Function or Gaussian) covariance matrix for the given input points. The
covariance between two points x_i and x_j is given by:

k(x_i, x_j) = α² * exp(-(x_i - x_j)² / (2ρ²)) + δ_ij * offset

where δ_ij is the Kronecker delta (1 if i=j, 0 otherwise).
"""
function cov_exp_quad(
    x::AbstractVector,
    α::Real,
    ρ::Real;
    offset::Real=1e-10
)
    # Compute pairwise squared distances
    dist_matrix = Distances.pairwise(Distances.SqEuclidean(), x)

    # Compute covariance matrix
    cov_matrix = @. α^2 * exp(-dist_matrix / (2ρ^2))

    # Add offset to diagonal for numerical stability
    cov_matrix[LinearAlgebra.diagind(cov_matrix)] .+= offset

    return cov_matrix
end

@doc raw"""
    cov_exp_quad(x₁, x₂, α, ρ)

Compute the Exponentiated Quadratic (RBF) covariance matrix between two sets of
points.

# Arguments
- `x₁::AbstractVector`: First set of points.
- `x₂::AbstractVector`: Second set of points.
- `α::Real`: Marginal standard deviation (output scale).
- `ρ::Real`: Length-scale parameter.

# Optional Keyword Arguments
- `offset::Real=1e-10`: Small value added to the diagonal for numerical
  stability.

# Returns
- `Matrix`: The computed cross-covariance matrix.

# Description
This function computes the Exponentiated Quadratic (also known as Radial Basis
Function or Gaussian) covariance matrix between two sets of points. The
covariance between points x₁_i and x₂_j is given by:

k(x₁_i, x₂_j) = α² * exp(-(x₁_i - x₂_j)² / (2ρ²))
"""
function cov_exp_quad(
    x₁::AbstractVector,
    x₂::AbstractVector,
    α::Real,
    ρ::Real,
    offset::Real=1e-10
)
    # Compute pairwise squared distances
    dist_matrix = Distances.pairwise(Distances.SqEuclidean(), x₁, x₂)

    # Compute covariance matrix
    cov_matrix = @. α^2 * exp(-dist_matrix / (2ρ^2))

    # Add offset to diagonal for numerical stability
    cov_matrix[LinearAlgebra.diagind(cov_matrix)] .+= offset

    return cov_matrix
end

## Prior distribution

Since we chose to use the exponential quadratic kernel, our prior distribution
over $f(x_*)$ now takes the form $\pi(f \mid \alpha, \rho)$. Given that $f$ is a
Gaussian process, the prior should be a Gaussian distribution of the form
$$
f(x_*) \sim \mathcal{N}(0, k(x_*, x_*')).
\tag{6}
$$
Let's sample from this prior distribution to get a better intuition for how both
hyper parameters affect the outcome.

In [12]:
Random.seed!(42)

# Define number of time points
n_points = 300
# Define time points to evaluate
x_array = collect(LinRange(-3, 3, n_points))

# Define single marginal standard deviation
α = 1.0

# Define different length scales
ρ = [1.0, 0.5, 0.25]

# Inititalize array to save samples
y = Array{Float64}(undef, length(ρ), n_points)

# Loop through length scales
for (i, r) in enumerate(ρ)
    # Build covariance matrix
    K = cov_exp_quad(x_array, α, r)

    # Generate samples
    y[i, :] = rand(Distributions.MvNormal(zeros(n_points), K))
end # for

Let's take a look at this synthetic data.

In [ ]:
# Inititalize figure
fig = Figure(size=(400, 300))

# Add axis
ax = Axis(
    fig[1, 1],
    xlabel="x",
    ylabel="f(x)"
)

# Loop through each random sample
for (i, y_) in enumerate(eachrow(y))
    scatter!(
        ax,
        x_array,
        y_,
        color=ColorSchemes.seaborn_colorblind[i],
        label="ρ = $(ρ[i])",
        markersize=6,
    )
end # for

# Add legend to plot
Legend(fig[1, 2], ax)

fig

From this we can clearly see the effect of the length scale parameter. Let's now draw many many samples using the same set of hyperparameters

In [ ]:
Random.seed!(42)

# Define number of points
n_points = 200
# Define time points to evaluate
x_array = collect(LinRange(-3, 3, n_points))

# Set number of samples
n_samples = 500

# Define hypeerparameters
α = 1.0
ρ = 1.0

# Compute covariance matrix
K = cov_exp_quad(x_array, α, ρ)

# Generate samples
samples = rand(Distributions.MvNormal(zeros(n_points), K), n_samples)

# Inititalize figure
fig = Figure(size=(450, 300))

# Add axis
ax = Axis(
    fig[1, 1],
    xlabel="x",
    ylabel="f(x)"
)

# Loop through each random sample
for (i, y_) in enumerate(eachcol(samples))
    lines!(
        ax,
        x_array,
        y_,
        color=(ColorSchemes.seaborn_colorblind[1], 0.15),
    )
end # for

fig

So far we have only discussed the prior $\pi(f(x_*) \mid x_*, \alpha, \rho)$.
But our choice of covariance kernel demands us to define priors for the
hyperparameters themselves. Once we implement the actual inference in
`Turing.jl` we'll discuss more the form that the priors $\pi(\alpha)$ and
$\pi(\rho)$ will take.

## Likelihood function

The assumption for our Gaussian process is that we expect our observable optical
density $y$ to follow the resulting function $f(x_*)$, perhaps with some error,
i.e.
$$
y_i = f(x_i) + \epsilon_i,
\tag{7}
$$
where $\epsilon_i$ is the deviation of the $i^{th}$ point from the Gaussian
process. We can then model the observable as a Normally distributed random
variable with uniform error $\sigma$,
$$
y_i \sim \mathcal{N}(f(x_i), \sigma).
\tag{8}
$$
This particular choice of likelihood has the very convenient property of being
the **conjugate likelihood** to a Gaussian process prior. What this means is
that the posterior distribution for $f(x_*)$ takes the same functional form as
the prior, i.e.,
$$
f(x_*) \mid y \sim 
\mathcal{GP}(m(x_*), k(x_*, x_*') + \delta_{x_*, x_*'}\, \sigma^2),
\tag{9}
$$
where $\delta_{x_*, x_*'}$ is the Kronecker delta. Let's quickly take a look at
what this implies. For $x_* \neq x_*'$ our mean function $m(x_*)$ will be zero
given our previous assumption. The covariance function for this case takes the
form of the usual exponential quadratic kernel given that $\delta_{x_*, x_*'} =
0$ for $x_* \neq t'$. For the specific case where $t = t'$ the quadratic kernel
for this case would be zero. But given our assumption of our measurements $y$
being normally distributed, the error in the measurement $\sigma$ now enters the
problem.

When we have multiple observations, for example the time series we obtain for
growth curves $\mathbf{x} = [x_0, x_1, \ldots, x_n]$, we define
$$
\mathbf{K}_y \equiv K(\mathbf{x}, \mathbf{x}) + \sigma^2 \mathbf{I},
\tag{10}
$$
where $\mathbf{I}$ is the identity matrix. Then we have a likelihood function of
the form
$$
f(\mathbf{x}) \sim \mathcal{N}(\mathbf{0}, \mathbf{K}_y).
\tag{11}
$$

# Bayesian fit

Now, we are ready to declare the `Turing` model using the `@model` macro. We
will define the following priors for the $\alpha$, $\rho$, and $\sigma$
parameters:
$$
\alpha \sim \mathcal{N}(\mu_\alpha, \sigma_\alpha),
\tag{12}
$$
and
$$
\rho \sim \text{Lognormal}(\mu_\rho, \sigma_\rho),
\tag{13}
$$
and
$$
\sigma \sim \text{Lognormal}(\mu_\sigma, \sigma_\sigma),
\tag{14}
$$

In [ ]:
# Declare our turing model
Turing.@model function logistic_gp(
    x, y, α_prior, ρ_prior, σ_prior
)
    # Priors
    α ~ Turing.LogNormal(α_prior...)
    ρ ~ Turing.LogNormal(ρ_prior...)
    σ ~ Turing.LogNormal(σ_prior...)

    # Compute Exponentiated quadratic covariance function
    cov_exp = cov_exp_quad(x, α, ρ; offset=1E-10) +
              (LinearAlgebra.I(length(x)) * σ^2)

    # Check if covariance matrix is positive definite. If not, set probability
    # to -∞. NOTE: I tried everything to make sure the matrix was positive 
    # definite, but I couldn't get it to work. This is a way to get around the
    # problem
    if !LinearAlgebra.isposdef(cov_exp)
        Turing.@addlogprob! -Inf
        return
    end

    # Likelihood
    return y ~ Turing.MvNormal(zeros(length(x)), cov_exp)
end # @model function

Now we are ready to run the sampler. Specifically we will use the `NUTS`
algorithm.

In [ ]:
Random.seed!(42)
# Group data by antibiotic, environment, and day
df_group = DF.groupby(
    df[(.!df.blank).&(df.concentration_ugmL.>0), :],
    [:antibiotic, :env, :day, :strain_num]
)

# Extract data
data = df_group[42]

# Extract data
x = log.(data.concentration_ugmL)
y = log.(data.OD)

# Define model
model = logistic_gp(x, y, [0.0, 1.0], [-0.4, 0.4], [0.0, 1.0])

# Define number of steps
n_burnin = 2_000
n_samples = 1_000

chain = Turing.sample(
    model, Turing.NUTS(), Turing.MCMCThreads(), n_burnin + n_samples, 4
)

# Remove burnin
chain = chain[n_burnin+1:end]

Turing.summarystats(chain)

Let's look at the trace and density plots for the parameters.

In [ ]:
# Extract parameters
params = names(chain, :parameters)

# Extract number of chains
n_chains = size(chain, 3)
# Extract number of samples
n_samples = size(chain, 1)

# Initialize figure
fig = Figure(size=(600, 400))

# Define colors
colors = ColorSchemes.seaborn_colorblind

# Loop through parameters
for (i, param) in enumerate(params)
    # Add axis for chain iteration
    ax_trace = Axis(fig[i, 1]; ylabel=string(param))
    # Inititalize axis for density plot
    ax_density = Axis(fig[i, 2]; ylabel=string(param))
    # Loop through chains
    for chn in 1:n_chains
        # Extract values
        values = chain[:, param, chn]
        # Plot traces of walker
        lines!(
            ax_trace,
            1:n_samples,
            values;
            label=string(chain),
            color=(colors[chn], 0.3)
        )
        # Plot density
        density!(
            ax_density, values; label=string(chain), color=(colors[chn], 0.3)
        )
    end # for

    # Hide y-axis decorations
    hideydecorations!(ax_trace; label=false)
    hideydecorations!(ax_density; label=false)

    # Check if it is bottom plot
    if i < length(params)
        # hide x-axis decoratiosn
        hidexdecorations!(ax_trace; grid=false)
    else
        # add x-label
        ax_trace.xlabel = "iteration"
        ax_density.xlabel = "parameter estimate"
    end # if
end # for

fig

Now, we have everything in place to compute the posterior predictive checks for
the growth curve. This is the "unique" part of the implementation, where, on top
of making predictions for unobserved time points (interpolating between the data
with as much resolution as desired), we will also infer the derivative of the
measured time series.

The math behind this is a little tricky and will be explained elsewhere.

First, we define a function to generate posterior predictive checks (PPC) for a
Gaussian process, including both function values and derivatives.

In [ ]:
@doc raw"""
    gp_ppc_rng(x_ppc, y, x_data, α, ρ, σ; offset=1E-10)

Generate posterior predictive checks (PPC) for a Gaussian process, including
both function values and derivatives.

# Arguments
- `x_ppc::AbstractVector`: Points where to evaluate PPC.
- `y::AbstractVector`: Observed measurements.
- `x_data::AbstractVector`: Observation points.
- `α::Real`: Marginal standard deviation.
- `ρ::Real`: Length scale.
- `σ::Real`: Measurement error standard deviation.
- `offset::Real=1E-10`: Small value added for numerical stability.

# Returns
- `Vector`: Concatenated vector of PPC samples for function values and
  derivatives.
"""
function gp_ppc_rng(
    x_ppc::AbstractVector,
    y::AbstractVector,
    x_data::AbstractVector,
    α::Real,
    ρ::Real,
    σ::Real;
    offset::Real=1E-10
)
    N_data, N_ppc = length(y), length(x_ppc)

    # Build necessary covariance matrices
    ## 1. Build bottom left covariance matrix K₂₂
    ### 1.1 Build Kx*x*
    K_xs_xs = cov_exp_quad(x_ppc, α, ρ; offset)

    ### 1.2 Initialize and compute d1x_Kx*x*, d2x_Kx*x*, and dxx_Kx*x*
    d1x_K_xs_xs = similar(K_xs_xs)
    d2x_K_xs_xs = similar(K_xs_xs)
    d2xx_K_xs_xs = similar(K_xs_xs)

    ### 1.4 Compute derivatives of the matrices by multiplying by corresponding
    ### prefactors
    @inbounds for i in 1:N_ppc, j in 1:N_ppc
        diff = x_ppc[i] - x_ppc[j]
        diff_sq = diff^2
        factor = K_xs_xs[i, j] / ρ^2
        d1x_K_xs_xs[i, j] = -diff * factor
        d2x_K_xs_xs[i, j] = diff * factor
        d2xx_K_xs_xs[i, j] = (1 - diff_sq / ρ^2) * factor
    end

    ### 1.5 Concatenate matrices
    K_22 = [K_xs_xs d1x_K_xs_xs'; d1x_K_xs_xs d2xx_K_xs_xs]

    ## 2. Compute top right and bottom left matrices K₁₂, K₂₁
    ### 2.1 Build Kxx*
    K_x_xs = cov_exp_quad(x_data, x_ppc, α, ρ)
    K_xs_x = cov_exp_quad(x_ppc, x_data, α, ρ)

    ### 2.2 Initialize and compute d1x_Kx*x and d2x_Kxx*
    d2x_K_x_xs = Matrix{Float64}(undef, N_data, N_ppc)
    d1x_K_xs_x = Matrix{Float64}(undef, N_ppc, N_data)

    ### 2.3 Compute derivative of matrices by multiplying by corresponding
    ### prefactors
    @inbounds for i in 1:N_data, j in 1:N_ppc
        d2x_K_x_xs[i, j] = (x_data[i] - x_ppc[j]) * K_x_xs[i, j] / ρ^2
        d1x_K_xs_x[j, i] = -(x_ppc[j] - x_data[i]) * K_xs_x[j, i] / ρ^2
    end

    ### 2.5 Concatenate matrices
    K_12 = [K_x_xs d2x_K_x_xs]
    K_21 = [K_xs_x; d1x_K_xs_x]

    ## 3. Solve equation Kxx * a = y
    ### 3.1 Generate covariance matrix for the data Kxx
    K_x_x = cov_exp_quad(x_data, α, ρ) .+ LinearAlgebra.I(N_data) * σ^2

    ### 3.2 Perform Cholesky decomposition Kxx = Lxx * Lxx'
    L_x_x = LinearAlgebra.cholesky(K_x_x).L

    ### 3.3 Solve for b = inv(Lxx) y taking advantage that Lxx is a triangular
    ### matrix
    b = L_x_x \ y

    ### 3.4 Solve a = inv(Lxx') b taking advantage that Lxx is a triangular
    ### matrix. Recall that a = inv(Kxx) y
    a = L_x_x' \ b

    ## 4. Compute conditional mean ⟨[f(x*), dx*f(x*)] | f(x)⟩
    mean_conditional = K_21 * a

    ## 5. Evaluate v = inv(Lxx) * Kxx*
    v = L_x_x \ K_12

    ## 6. Evaluate v' = inv(Lxx) * Kx*x
    v_prime = (L_x_x \ K_21')'

    ## 7. Compute conditional covariance
    cov_conditional = K_22 - v_prime * v .+ LinearAlgebra.I(2 * N_ppc) * offset

    # Generate random samples given the conditional mean and covariance
    return rand(Distributions.MvNormal(
        mean_conditional, LinearAlgebra.Symmetric(cov_conditional)
    ))
end # function

In [ ]:
@doc raw"""
    gp_ppc_samples(chain, x_ppc, y, x_data; offset=1E-10, multithread=true)

Generate posterior predictive checks (PPC) for the experimental data and its
unobserved derivative from a Gaussian process, given MCMC chain samples.

# Arguments
- `chain::MCMCChains.Chains`: Samples from the MCMC run generated with
  Turing.jl. The chain is assumed to have three parameters in the following
  order:
    - `α::AbstractFloat`: Marginal standard deviation.
    - `ρ::AbstractFloat`: scale.
    - `σ::AbstractFloat`: Measurement error. 
- `x_ppc::AbstractVector`: Points where to evaluate PPC.
- `y::AbstractVector`: Observed measurements.
- `x_data::AbstractVector`: Observation points.
- `offset::AbstractFloat=1E-10`: Extra value added for numerical stability.
- `multithread::Bool=true`: Whether to use multithreading for computations.

# Returns
- `y_predict::Array{Float64, 3}`: Predictions for the observation process.
- `dy_predict::Array{Float64, 3}`: Predictions for the derivative of the
  observation process.
"""
function gp_ppc_samples(
    chain::MCMCChains.Chains,
    x_ppc::AbstractVector,
    y::AbstractVector,
    x_data::AbstractVector;
    offset::AbstractFloat=1E-10,
    multithread::Bool=true
)
    # Get the number of prediction points
    N_predict = length(x_ppc)
    # Get the parameter names from the chain
    params = names(chain, :parameters)
    # Get the number of chains
    n_chains = size(chain, 3)
    # Get the number of samples
    n_samples = size(chain, 1)
    # Extract the parameter samples from the chain
    chain_samples = MCMCChains.get_params(chain)

    # Initialize arrays to store predictions
    f_predict = Array{Float64}(undef, n_samples, n_chains, 2 * N_predict)
    y_predict = Array{Float64}(undef, n_samples, n_chains, N_predict)
    dy_predict = Array{Float64}(undef, n_samples, n_chains, N_predict)

    # Define a function to process each sample
    process_sample = (i, j) -> begin
        # Extract parameter values for this sample
        α_, ρ_, σ_ = (chain_samples[params[k]].data[i, j] for k in 1:3)
        # Generate predictions using the GP model
        f_predict[i, j, :] = gp_ppc_rng(
            x_ppc, y, x_data, α_, ρ_, σ_; offset=offset
        )
        for n in 1:N_predict
            # Generate noisy observations from the GP predictions
            y_predict[i, j, n] = rand(
                Distributions.Normal(f_predict[i, j, n], σ_)
            )
            # Generate derivative predictions
            dy_predict[i, j, n] = rand(
                Distributions.Normal(f_predict[i, j, N_predict+n], offset)
            )
        end
    end

    # Process samples using multithreading if enabled
    if multithread
        Threads.@threads for i in 1:n_samples
            for j in 1:n_chains
                process_sample(i, j)
            end
        end
    else
        # Process samples sequentially if multithreading is disabled
        for i in 1:n_samples, j in 1:n_chains
            process_sample(i, j)
        end
    end

    # Return the predictions for observations and derivatives
    return y_predict, dy_predict
end

With these functions in place, we can now generate the posterior predictive
checks for the growth curves.


In [ ]:
# Define number of points to predict
n_points = 100
# Define time points to evaluate
x_array = collect(LinRange(minimum(x), maximum(x), n_points))

# Generate posterior predictive checks
y_predict, dy_predict = gp_ppc_samples(chain, x_array, y, x)

To plot the generated samples in a sensible way, let's compute a few quantiles
that we'll display with different shades of color.

In [69]:
# Define quantiles to compute
qs = [0.95, 0.68, 0.5]

# Inititalize array to save quantiles
y_quant = [Array{Float64}(undef, size(y_predict, 3), 2) for x = 1:length(qs)]
dy_quant = [Array{Float64}(undef, size(y_predict, 3), 2) for x = 1:length(qs)]

# Loop through quantiles
for (i, q) in enumerate(qs)
    # Loop through time points
    for j = 1:size(y_predict, 3)
        # Flatten data
        y_data = vec(y_predict[:, :, j])
        dy_data = vec(dy_predict[:, :, j])

        # Compute uper and lower quantile
        y_quant[i][j, :] = StatsBase.quantile(
            y_data, [(1.0 - q) / 2.0, 1.0 - (1.0 - q) / 2.0]
        )

        dy_quant[i][j, :] = StatsBase.quantile(
            dy_data, [(1.0 - q) / 2.0, 1.0 - (1.0 - q) / 2.0]
        )
    end # for
end # for

Now, we are ready to plot the inferred time series along with the derivative.

In [ ]:
# Initialize figure
fig = Figure(size=(350, 500))

# Add ax
ax1 = Axis(
    fig[1, 1],
    xlabel="log(concentration)",
    ylabel="log(OD)"
)

ax2 = Axis(
    fig[2, 1],
    xlabel="log(concentration)",
    ylabel="d(log(OD))/dt"
)

# Define colors
colors = get(ColorSchemes.Blues_9, LinRange(0, 0.75, 3))

# Loop through quantiles
for (i, q) in enumerate(qs)
    # Add confidence interval for observation
    band!(
        ax1,
        x_array,
        y_quant[i][:, 1],
        y_quant[i][:, 2],
        color=(colors[i], 0.75)
    )

    # Add confidence interval for derivative
    band!(
        ax2,
        x_array,
        dy_quant[i][:, 1],
        dy_quant[i][:, 2],
        color=(colors[i], 0.75)
    )
end # for

# Add growth curve scatter plot
scatter!(ax1, x, y, markersize=5, color=:black)

fig

In [ ]:
function logistic(logx, params)
    return logistic(logx, params...)
end

"""
    fit_logistic_and_detect_outliers(log2x, y; threshold=3)

Fit a logistic model to the given data and detect outliers based on residuals.

This function performs the following steps:
1. Fits a logistic model to the log2-transformed x-values and y-values.
2. Calculates residuals between the fitted model and actual y-values.
3. Identifies outliers as points with residuals exceeding a specified threshold.

# Arguments
- `log2x`: Array of log2-transformed x-values (typically concentrations).
- `y`: Array of y-values (typically optical density measurements).
- `threshold`: Number of standard deviations beyond which a point is considered
  an outlier. Default is 3.

# Returns
- A boolean array indicating which points are outliers (true for outliers).

# Notes
- The function uses a logistic model of the form: y = a / (1 + exp(b * (log2x -
    log2ic50))) + c
- Initial parameter guesses are made based on the input data.
- The LsqFit package is used for curve fitting.
- Outliers are determined by comparing the absolute residuals to the threshold *
  standard deviation of residuals.
"""
function fit_logistic_and_detect_outliers(logx, logy; threshold=3)
    # Initial parameter guess
    p0 = [0.1, 1.0, maximum(logy) - minimum(logy), StatsBase.median(logx)]

    # Fit the logistic model
    fit = LsqFit.curve_fit(logistic, logx, logy, p0)

    # Calculate residuals
    residuals = logy - logistic(logx, fit.param)

    # Calculate standard deviation of residuals
    σ = std(residuals)

    # Identify outliers
    outliers_idx = abs.(residuals) .> threshold * σ

    # Return outlier indices
    return outliers_idx
end

In [ ]:
Random.seed!(42)
# Group data by antibiotic, environment, and day
df_group = DF.groupby(
    df[(.!df.blank).&(df.concentration_ugmL.>0), :],
    [:antibiotic, :env, :day, :strain_num]
)

# Extract data
data = df_group[42]

# Find outliers
outliers_idx = fit_logistic_and_detect_outliers(
    log.(data.concentration_ugmL), log.(data.OD), threshold=2
)
# Remove outliers
data_clean = data[.!outliers_idx, :]

# Extract data
x = log.(data_clean.concentration_ugmL)
y = log.(data_clean.OD)

# Define model
model = logistic_gp(x, y, [0.0, 1.0], [-0.4, 0.4], [0.0, 1.0])

# Define number of steps
n_burnin = 2_500
n_samples = 500

chain = Turing.sample(
    model, Turing.NUTS(), Turing.MCMCThreads(), n_burnin + n_samples, 4
)

In [73]:
# Define number of points to predict
n_points = 100
# Define time points to evaluate
x_array = collect(LinRange(minimum(x), maximum(x), n_points))

# Generate posterior predictive checks
y_predict, dy_predict = gp_ppc_samples(chain, x_array, y, x)

# Define quantiles to compute
qs = [0.95, 0.68, 0.5]

# Inititalize array to save quantiles
y_quant = [Array{Float64}(undef, size(y_predict, 3), 2) for x = 1:length(qs)]
dy_quant = [Array{Float64}(undef, size(y_predict, 3), 2) for x = 1:length(qs)]

# Loop through quantiles
for (i, q) in enumerate(qs)
    # Loop through time points
    for j = 1:size(y_predict, 3)
        # Flatten data
        y_data = vec(y_predict[:, :, j])
        dy_data = vec(dy_predict[:, :, j])

        # Compute uper and lower quantile
        y_quant[i][j, :] = StatsBase.quantile(
            y_data, [(1.0 - q) / 2.0, 1.0 - (1.0 - q) / 2.0]
        )

        dy_quant[i][j, :] = StatsBase.quantile(
            dy_data, [(1.0 - q) / 2.0, 1.0 - (1.0 - q) / 2.0]
        )
    end # for
end # for